# Llama2Chat

This notebook shows how to augment Llama-2 `LLM`s with the `Llama2Chat` wrapper to support the [Llama-2 chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2). Several `LLM` implementations in LangChain can be used as interface to Llama-2 chat models. These include [ChatHuggingFace](/docs/integrations/chat/huggingface), [LlamaCpp](/docs/use_cases/question_answering/local_retrieval_qa), [GPT4All](/docs/integrations/llms/gpt4all), ..., to mention a few examples. 

`Llama2Chat` is a generic wrapper that implements `BaseChatModel` and can therefore be used in applications as [chat model](/docs/modules/model_io/chat/). `Llama2Chat` converts a list of Messages into the [required chat prompt format](https://huggingface.co/blog/llama2#how-to-prompt-llama-2) and forwards the formatted prompt as `str` to the wrapped `LLM`.

In [1]:
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_experimental.chat_models import Llama2Chat

For the chat application examples below, we'll use the following chat `prompt_template`:

In [2]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
)

template_messages = [
    SystemMessage(content="You are a helpful assistant."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("{text}"),
]
prompt_template = ChatPromptTemplate.from_messages(template_messages)

In [3]:
# !pip3 install text-generation

## Chat with Llama-2 via `LlamaCPP` LLM

For using a Llama-2 chat model with a [LlamaCPP](/docs/integrations/llms/llamacpp) `LMM`, install the `llama-cpp-python` library using [these installation instructions](/docs/integrations/llms/llamacpp#installation). The following example uses a quantized [llama-2-7b-chat.Q4_0.gguf](https://huggingface.co/TheBloke/Llama-2-7b-Chat-GGUF/resolve/main/llama-2-7b-chat.Q4_0.gguf) model stored locally at `~/Models/llama-2-7b-chat.Q4_0.gguf`. You can download it from  [here](https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGUF/blob/main/llama-2-7b-chat.Q4_0.gguf)

After creating a `LlamaCpp` instance, the `llm` is again wrapped into `Llama2Chat`

In [4]:
from langchain_community.llms import LlamaCpp
model_path = "/home/ashok/Models/llama-2-7b-chat.Q4_0.gguf"
llm = LlamaCpp(
                model_path=model_path,
                streaming=False,
                )

model = Llama2Chat(llm=llm)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /home/ashok/Models/llama-2-7b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u3

and used in the same way as in the previous example.

In [5]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chain = LLMChain(llm=model, prompt=prompt_template, memory=memory)

In [9]:
msg=    chain.invoke(
                  "What can I see in Vienna? Propose a few locations. Names only, no details."
                    )


Llama.generate: prefix-match hit

llama_print_timings:        load time =     783.31 ms
llama_print_timings:      sample time =      46.65 ms /    96 runs   (    0.49 ms per token,  2057.79 tokens per second)
llama_print_timings: prompt eval time =    9389.07 ms /   125 tokens (   75.11 ms per token,    13.31 tokens per second)
llama_print_timings:        eval time =   15684.02 ms /    95 runs   (  165.09 ms per token,     6.06 tokens per second)
llama_print_timings:       total time =   25391.57 ms /   220 tokens


In [11]:
print(msg['text'])

  Of course! Here are some popular tourist attractions and landmarks in Vienna:
1. Schönbrunn Palace
2. St. Stephen's Cathedral
3. Hofburg Palace
4. Belvedere Palace
5. Prater Amusement Park
6. Vienna State Opera
7. MuseumsQuartier
8. Ringstrasse
9. St. Charles Borromeo Church
10. Imperial Burial Chapel


In [12]:
msg1 = chain.invoke("Tell me more about #2.")

Llama.generate: prefix-match hit

llama_print_timings:        load time =     783.31 ms
llama_print_timings:      sample time =     112.97 ms /   228 runs   (    0.50 ms per token,  2018.22 tokens per second)
llama_print_timings: prompt eval time =    8905.29 ms /   112 tokens (   79.51 ms per token,    12.58 tokens per second)
llama_print_timings:        eval time =   38896.00 ms /   227 runs   (  171.35 ms per token,     5.84 tokens per second)
llama_print_timings:       total time =   48545.35 ms /   339 tokens


In [13]:
print(msg1['text'])

  Certainly! St. Stephen's Cathedral is a beautiful and historic church located in the heart of Vienna, Austria. It is one of the most famous and recognizable landmarks in the city and is known for its stunning Gothic architecture.
The cathedral was built in the 12th century and has undergone several renovations and expansions over the centuries. It features a striking mix of Gothic and Baroque styles, with intricate carvings, colorful stained-glass windows, and a towering spire that offers panoramic views of the city.
Inside the cathedral, visitors can admire the ornate decorations, including the tomb of St. Stephen, the patron saint of Vienna, as well as works of art by famous artists such as Hans Holbein the Younger and Lucas Cranach the Elder. The cathedral's organ is also worth mentioning, as it is one of the largest and most powerful in Europe, with over 17,000 pipes.
St. Stephen'
